In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.models import PrositIntensityPredictor
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
print([x for x in dir(dlomix) if not x.startswith("_")])

2023-05-23 12:10:19.142272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 12:10:19.898748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


['META_DATA', 'constants', 'data', 'eval', 'layers', 'losses', 'models', 'pipelines', 'reports', 'utils']


In [15]:
from dlomix.constants import ALPHABET_UNMOD
ALPHABET_UNMOD

{'A': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'K': 9,
 'L': 10,
 'M': 11,
 'N': 12,
 'P': 13,
 'Q': 14,
 'R': 15,
 'S': 16,
 'T': 17,
 'V': 18,
 'W': 19,
 'Y': 20}

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
project_name = "transforming-prosit"

In [18]:
df_annotation = pd.read_parquet('/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_59_01_01_annotation.parquet')

In [19]:
df_annotation.head()

,ion_type,no,charge,experimental_mass,theoretical_mass,intensity,neutral_loss,fragment_score,peptide_sequence,scan_number,raw_file
0,b,6,2,266.15887,266.105904,0.03,C3H5NO-C3H6,39,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
1,b,6,2,284.12894,283.632089,0.05,C2H4O2-H2O,64,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
2,b,6,2,284.12894,284.124097,0.05,C2H4O2-NH3,64,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
3,b,6,2,284.12894,284.124097,0.05,CH3NO-CH4O,39,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
4,b,6,2,284.12894,284.124097,0.05,C2H5NO-H2O,64,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2


In [16]:
df = pd.read_parquet('/cmnfs/proj/prosit/Transformer/Final_Meta_Data/TUM_aspn_meta_data_sum_intensities_per_raw_file.parquet')


In [20]:
df.head()

,raw_file,scan_number,modified_sequence,precursor_charge,precursor_intensity,mz,precursor_mz,fragmentation,mass_analyzer,retention_time,indexed_retention_time,andromeda_score,peptide_length,orig_collision_energy,aligned_collision_energy,Raw file,y_b_sum_intensities
0,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,39748,DSVGEITKEDLIQKSLGTC[UNIMOD:4]Q,2,7552496.0,1111.0545,1111.055135,CID,ITMS,38.778,65.467590,396.05,20,35.0,35.000000,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,12.417911
1,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,39777,DSVGEITKEDLIQKSLGTC[UNIMOD:4]Q,2,7674974.0,1111.0545,1111.054935,CID,ITMS,38.798,65.517586,367.48,20,35.0,35.000000,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,12.436208
2,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,39677,DSVGEITKEDLIQKSLGTC[UNIMOD:4]Q,2,3298128.0,1111.0545,1111.055336,CID,ITMS,38.729,65.345099,351.16,20,35.0,35.000000,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,11.625682
3,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,40022,DSVGEITKEDLIQKSLGTC[UNIMOD:4]Q,2,1224600.0,1111.0545,1111.055135,CID,ITMS,38.968,65.942554,301.38,20,35.0,35.000000,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,10.994842
4,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,39678,DSVGEITKEDLIQKSLGTC[UNIMOD:4]Q,2,3298128.0,1111.0545,1111.055336,HCD,ITMS,38.730,65.347599,295.53,20,28.0,30.113057,03210a_BE5-TUM_aspn_5_01_01-2xIT_2xHCD-1h-R1,11.022819
